# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-02-24 18:22:33] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-02-24 18:22:33] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-02-24 18:22:33] INFO utils.py:164: NumExpr defaulting to 16 threads.


<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1184: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.


[2026-02-24 18:22:36] INFO server_args.py:1828: Attention backend not specified. Use fa3 backend by default.


...

Capturing batches (bs=64 avail_mem=15.10 GB):  40%|████      | 8/20 [00:00<00:00, 14.82it/s]

Capturing batches (bs=40 avail_mem=15.08 GB):  50%|█████     | 10/20 [00:00<00:00, 14.17it/s]

Capturing batches (bs=16 avail_mem=15.06 GB):  70%|███████   | 14/20 [00:01<00:00, 14.40it/s]

Capturing batches (bs=4 avail_mem=15.05 GB):  80%|████████  | 16/20 [00:01<00:00, 11.46it/s]

Capturing batches (bs=1 avail_mem=15.04 GB): 100%|██████████| 20/20 [00:01<00:00, 13.44it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Ian and I'm a software engineer in New York City. I've been working on the development of a number of new programming languages, and have also worked on frameworks and libraries that enable the creation and use of these languages. I'm passionate about the development of software and want to continue to learn about it. My current programming language is Python.

I am interested in the history of computers and how they have evolved over time, and I am interested in the impact of technology on society. I am also interested in the creation and use of software and technology.

Can you provide some insights on the impact of technology on society and the creation and use
Prompt: The president of the United States is
Generated text:  elected by ______.
A. the people to elect them.
B. the country's parliament to elect them.
C. the president to elect them.
D. the nation's two houses to elect them.
Answer:
A

What is the purpose of an evaluation activity

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? [Name] is a [job title] at [company name]. I'm excited to meet you and learn more about

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. 

The statement is concise and accurately describes the capital city of France. It is a factual statement that provides a clear and specific answer to the question asked. The statement is concise because it only mentions the capital city of France, while still providing a complete and accurate answer. The statement is accurate because it is based on historical and official information about the capital city of France. The statement is concise and accurate because it is a single sentence that does not contain any unnecessary words or phrases. The statement is concise and accurate because it is a single sentence that does not contain any unnecessary words or phrases. The statement is concise and accurate because it

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends that could be expected in the future:

1. Increased integration of AI into everyday life: As AI becomes more integrated into our daily lives, we are likely to see more widespread adoption of AI technologies. This could include things like voice assistants, self-driving cars, and chatbots that can understand and respond to human language.

2. Greater emphasis on ethical AI: As AI becomes more integrated into our daily lives, there will be a greater emphasis on ethical considerations. This could include things like



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a [Occupation/Job] with [Number of Years] years of experience in [Field/Industry]. I've always been passionate about [What excites you about your job], and I believe that [Why you love what you do]. I'm always looking to learn and grow, so if you have any questions or suggestions, don't hesitate to reach out to me! How can I get to know you better? Welcome, [Name]. I'm happy to have you on my team and ready to help you achieve your goals! Let's get to know each other better. [Name]. [Name

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, located in the heart of the region of Paris. It is the largest city in the country and the second-largest in the European Union. Paris is known for its rich history, vibrant culture, and beautiful architecture, including the Eiffel Tower, Louvre Museum, an

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

].

...

 more

 integrated

 into

 more

 industries

In [6]:
llm.shutdown()